In [1]:
# @todo -> these are intermediate versions of the code because of testing!!
import time
from algorithm.helper import get_specific_station_identifier_from_name, get_graph_data




def setup_network_data(start, destination, start_time, use_example=False):
    if not use_example:
        # get stop ids from names
        start = get_specific_station_identifier_from_name(stop_name=start)
        destination = get_specific_station_identifier_from_name(stop_name=destination)

        # parse to number
        start = int(start)
        destination = int(destination)


    # set end time artificially (only support up to 6 hours)
    end_time = start_time + 180

    run_time_start_generate_graph = time.time()
    # get data
    graph = get_graph_data("2024-10-02", "../transport_data.db", start_time, end_time, start, use_example_data=use_example)
    logging.info("Graph data loaded")

    run_time_end_generate_graph = time.time()
    runtime_generate_graph = run_time_end_generate_graph - run_time_start_generate_graph
    logging.info(f"Runtime for generating graph: {runtime_generate_graph} seconds")

    return graph, start, destination


In [2]:

import logging

start_time = 600  # 10 AM, please set this to what you need
start = "Luzern"  #
destination = "Zug"  #
# Setup data
graph, start, destination = setup_network_data(start, destination, start_time, use_example=False)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:root:Data loaded for 2024-10-02
INFO:root:Graph data loaded
INFO:root:Runtime for generating graph: 76.92204737663269 seconds


In [3]:
from constants import LOG_LEVEL
LOG_LEVEL = logging.DEBUG
logging.basicConfig(level=LOG_LEVEL)
LOG_LEVEL

10

In [4]:


from algorithm.helper import print_path
from algorithm.reliability_v2 import compute_reliability
from algorithm.graph import Graph


def run_algorithms(graph, start, destination, start_time, time_budget_multiplier=1.5):
    # initialize graph G (with graph class)
    G = Graph(graph=graph)

    # if the graph is empty, return
    if not G.graph:
        logging.info("Graph is empty")
        return
    run_time_shortest_path_start = time.time()
    # find shortest path
    shortest_time, shortest_path = G.dijkstra(start, destination, start_time)
    run_time_shortest_path_end = time.time()
    runtime_shortest_path = run_time_shortest_path_end - run_time_shortest_path_start

    # check if length of shortest is 0, if so -> no shortest path found
    if len(shortest_path) == 0:
        return {}

    time_budget_shortest_path = (shortest_time - start_time) * 1  # for the shortest path, we have 100% (and not more) of the time budget
    shortest_path_reliability = compute_reliability(shortest_path, start_time, time_budget_shortest_path, transfer_time=5)
    logging.info("Found shortest path")
    print_path(shortest_path, start, start_time, convert_ids_to_names=True)

    # find most reliable path
    time_budget = time_budget_shortest_path * time_budget_multiplier
    run_time_start = time.time()
    reliable_arrival_time, reliability, most_reliable_path = G.find_most_reliable_path(start, destination, start_time, int(time_budget))
    run_time_end = time.time()
    runtime = run_time_end - run_time_start

    return {
        "earliest_arrival_time": shortest_time,
        "shortest_path": shortest_path,
        "shortest_path_reliability": shortest_path_reliability,
        "runtime_shortest_path": runtime_shortest_path,  # in seconds
        "most_reliable_path": most_reliable_path,
        "reliability": reliability,
        "runtime": runtime, # in seconds
        "start_station": start,
        "destination_station": destination
    }

In [5]:

# Run algorithms for shortest path (earliest possible arrival) and most reliable itinerary
time_budget_multiplier=2
result = run_algorithms(graph, start, destination, start_time, time_budget_multiplier)
print("**RESULT**")
result

INFO:root:Find shortest path from 8505000 to 8502204 starting at 600
DEBUG:root:No connection possible (shortest path)
DEBUG:root:No connection possible (shortest path)
DEBUG:root:***Found target 8502204
INFO:root:Found shortest path
INFO:root:Find most reliable path from 8505000 to 8502204 starting at 600 with a time budget of 114 minutes
DEBUG:root:Priority queue: [(0, 1, 0, None, [{'from': 8505000, 'to': 8505000, 'actual_times': []}])]
DEBUG:root:Priority queue: [(0.0, 2, 45, 0, [{'from': 8505000, 'to': 8505000, 'actual_times': []}, {'from': 8505000, 'planned_departure': 614, 'to': 8502021, 'planned_arrival': 619, 'trip_id': 'ch:1:sjyid:100001:21129-001', 'actual_times': [(614, 619), (614, 619), (615, 620), (614, 619), (614, 619), (614, 619), (614, 619), (614, 619), (614, 619), (614, 619), (615, 620), (614, 619), (616, 621), (614, 619), (616, 621), (614, 619), (616, 621), (614, 619), (2880, 2880), (614, 619), (615, 620), (614, 619), (614, 619), (615, 620), (614, 619)]}, {'from': 850

Start at Luzern at 600
Take S1 to Rotkreuz at 621 and arrive at 642
Take IR75 to Zug at 649 and arrive at 657


DEBUG:root:Priority queue: [(0.0, 3, 31, 0, [{'from': 8505000, 'to': 8505000, 'actual_times': []}, {'from': 8505000, 'planned_departure': 614, 'to': 8502021, 'planned_arrival': 619, 'trip_id': 'ch:1:sjyid:100001:21129-001', 'actual_times': [(614, 619), (614, 619), (615, 620), (614, 619), (614, 619), (614, 619), (614, 619), (614, 619), (614, 619), (614, 619), (615, 620), (614, 619), (616, 621), (614, 619), (616, 621), (614, 619), (616, 621), (614, 619), (2880, 2880), (614, 619), (615, 620), (614, 619), (614, 619), (615, 620), (614, 619)]}, {'from': 8505000, 'planned_departure': 618, 'to': 8505004, 'planned_arrival': 645, 'trip_id': 'ch:1:sjyid:100001:667-001', 'actual_times': [(619, 645), (618, 644), (620, 644), (618, 643), (622, 645), (619, 644), (619, 645)]}, {'from': 8505000, 'planned_departure': 621, 'to': 8502200, 'planned_arrival': 631, 'trip_id': 'ch:1:sjyid:100001:21130-001', 'actual_times': [(622, 630), (622, 630), (623, 630), (623, 630), (624, 634), (622, 630), (623, 630), (62

**RESULT**


{'earliest_arrival_time': 657,
 'shortest_path': [{'from': 8505000,
   'to': 8502202,
   'planned_departure': 621,
   'planned_arrival': 642,
   'trip_id': 'ch:1:sjyid:100001:21130-001',
   'actual_times': [(622, 643),
    (622, 642),
    (623, 642),
    (623, 643),
    (624, 644),
    (622, 643),
    (623, 646),
    (623, 643),
    (622, 642),
    (621, 643),
    (621, 642),
    (622, 642),
    (623, 643),
    (633, 653),
    (622, 642),
    (623, 645),
    (623, 642),
    (622, 642),
    (623, 644),
    (621, 643),
    (622, 641),
    (622, 643),
    (623, 642),
    (622, 644),
    (624, 643)]},
  {'from': 8502202,
   'to': 8502204,
   'planned_departure': 649,
   'planned_arrival': 657,
   'trip_id': 'ch:1:sjyid:100001:2217-001',
   'actual_times': [(649, 656),
    (649, 657),
    (649, 656),
    (649, 656),
    (649, 656),
    (650, 656),
    (651, 658),
    (649, 657),
    (650, 657),
    (649, 656),
    (649, 657),
    (649, 656),
    (649, 656),
    (652, 659),
    (651, 657),
 